# Setup

In [1]:
!pip install promptbench

In [2]:
!pip install textattack tensorflow tensorflow_hub

In [3]:
import promptbench as pb

from promptbench.models import LLMModel
from promptbench.prompt_attack import Attack
from promptbench.dyval import *

from tqdm import tqdm

In [4]:
class CFG:
    device = 'cuda'

# Basic functionality

In [5]:
print('All supported datasets: ')
print(pb.SUPPORTED_DATASETS)


All supported datasets: 

['sst2', 'cola', 'qqp', 'mnli', 'mnli_matched', 'mnli_mismatched', 'qnli', 'wnli', 'rte', 'mrpc', 'mmlu', 'squad_v2', 'un_multi', 'iwslt2017', 'math', 'bool_logic', 'valid_parentheses', 'gsm8k', 'csqa', 'bigbench_date', 'bigbench_object_tracking', 'last_letter_concat', 'numersense', 'qasc']


In [6]:
# load a dataset, sst2, for instance.
# if the dataset is not available locally, it will be downloaded automatically.
dataset = pb.DatasetLoader.load_dataset("sst2")
# dataset = pb.DatasetLoader.load_dataset("mmlu")
# dataset = pb.DatasetLoader.load_dataset("un_multi")
# dataset = pb.DatasetLoader.load_dataset("iwslt2017", ["ar-en", "de-en", "en-ar"])
# dataset = pb.DatasetLoader.load_dataset("math", "algebra__linear_1d")
# dataset = pb.DatasetLoader.load_dataset("bool_logic")
# dataset = pb.DatasetLoader.load_dataset("valid_parenthesesss")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 

The secret `HF_TOKEN` does not exist in your Colab secrets.

To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.

You will be able to reuse this secret in all of your notebooks.

Please note that authentication is recommended but still optional to access public models or datasets.

  warnings.warn(


In [7]:
# print the first 5 examples
dataset[:5]

[{'content': 'uneasy mishmash of styles and genres .', 'label': -1},
 {'content': "this film 's relationship to actual tension is the same as what christmas-tree flocking in a spray can is to actual snow : a poor -- if durable -- imitation .",
  'label': -1},
 {'content': 'by the end of no such thing the audience , like beatrice , has a watchful affection for the monster .',
  'label': -1},
 {'content': 'director rob marshall went out gunning to make a great one .',
  'label': -1},
 {'content': 'lathan and diggs have considerable personal charm , and their screen rapport makes the old story seem new .',
  'label': -1}]

Load models

In [8]:
# print all supported models in promptbench
print('All supported models: ')
print(pb.SUPPORTED_MODELS)


All supported models: 

['google/flan-t5-large', 'llama2-7b', 'llama2-7b-chat', 'llama2-13b', 'llama2-13b-chat', 'llama2-70b', 'llama2-70b-chat', 'phi-1.5', 'palm', 'gpt-3.5-turbo', 'gpt-4', 'gpt-4-1106-preview', 'gpt-3.5-turbo-1106', 'vicuna-7b', 'vicuna-13b', 'vicuna-13b-v1.3', 'google/flan-ul2']


In [9]:
model = pb.LLMModel(model='google/flan-t5-large', max_new_tokens=10, temperature=0.0001)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


construct a prompt by calling the Prompt API

In [10]:
prompts = pb.Prompt(["Classify the sentence as positive or negative: {content}",
                     "Determine the emotion of the following sentence as positive or negative: {content}"
                     ])




You may need to define the projection function for the model output. Since the output format defined in your prompts may be different from the model output. For example, for sst2 dataset, the label are '0' and '1' to represent 'negative' and 'positive'. But the model output is 'negative' and 'positive'. So we need to define a projection function to map the model output to the labe


In [11]:
def proj_func(pred):
    mapping = {
        "positive": 1,
        "negative": 0
    }
    return mapping.get(pred, -1)


In [12]:


for prompt in prompts:
    preds = []
    labels = []
    for data in tqdm(dataset):
        # process input
        input_text = pb.InputProcess.basic_format(prompt, data)
        label = data['label']
        raw_pred = model(input_text)
        # process output
        pred = pb.OutputProcess.cls(raw_pred, proj_func)
        preds.append(pred)
        labels.append(label)

    # evaluate
    score = pb.Eval.compute_cls_accuracy(preds, labels)
    print(f"{score:.3f}, {prompt}")



  0%|          | 0/1821 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0001` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.

  warnings.warn(

100%|██████████| 1821/1821 [03:02<00:00,  9.99it/s]


0.000, Classify the sentence as positive or negative: {content}


100%|██████████| 1821/1821 [03:01<00:00, 10.02it/s]

0.000, Determine the emotion of the following sentence as positive or negative: {content}


# Prompt engineering

In [13]:
# print all methods and their supported datasets
print('All supported methods: ')
print(pb.SUPPORTED_METHODS)


All supported methods: 

['CoT', 'ZSCoT', 'least_to_most', 'generated_knowledge', 'expert_prompting', 'emotion_prompt', 'baseline']


In [14]:


print('Supported datasets for each method: ')
print(pb.METHOD_SUPPORT_DATASET)


Supported datasets for each method: 

{'CoT': ['gsm8k', 'csqa', 'bigbench_date', 'bigbench_object_tracking'], 'ZSCoT': ['gsm8k', 'csqa', 'bigbench_date', 'bigbench_object_tracking'], 'expert_prompting': ['gsm8k', 'csqa', 'bigbench_date', 'bigbench_object_tracking'], 'emotion_prompt': ['gsm8k', 'csqa', 'bigbench_date', 'bigbench_object_tracking'], 'least_to_most': ['gsm8k', 'last_letter_concat'], 'generated_knowledge': ['csqa', 'numersense', 'qasc'], 'baseline': ['gsm8k', 'csqa', 'bigbench_date', 'bigbench_object_tracking', 'last_letter_concat', 'numersense', 'qasc']}


In [15]:
dataset_name = "csqa"
dataset = pb.DatasetLoader.load_dataset(dataset_name)

# print the first 5 examples
dataset[:5]


[{'content': 'A revolving door is convenient for two direction travel, but it also serves as a security measure at a what? \nAnswer Choices: (A) bank (B) library (C) department store (D) mall (E) new york',
  'label': 'A'},
 {'content': 'What do people aim to do at work? \nAnswer Choices: (A) complete job (B) learn from each other (C) kill animals (D) wear hats (E) talk to each other',
  'label': 'A'},
 {'content': 'Where would you find magazines along side many other printed works? \nAnswer Choices: (A) doctor (B) bookstore (C) market (D) train station (E) mortuary',
  'label': 'B'},
 {'content': 'Where are  you likely to find a hamburger? \nAnswer Choices: (A) fast food restaurant (B) pizza (C) ground up dead cows (D) mouth (E) cow carcus',
  'label': 'A'},
 {'content': 'James was looking for a good place to buy farmland.  Where might he look? \nAnswer Choices: (A) midwest (B) countryside (C) estate (D) farming areas (E) illinois',
  'label': 'A'}]

In [16]:
model = pb.LLMModel(model='google/flan-t5-large', max_new_tokens=10, temperature=0.0001)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [17]:

method = pb.PEMethod(method='CoT',
                        dataset=dataset_name,
                        verbose=True,  # if True, print the detailed prompt and response
                        prompt_id = 1  # for emotion_prompt
                        )

results = method.test(dataset,
                      model,
                      num_samples=5 # if don't set the num_samples, method will use all examples in the dataset
                      )

results


  0%|          | 0/1221 [00:00<?, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (655 > 512). Running this sequence through the model will result in indexing errors

  0%|          | 1/1221 [00:00<08:30,  2.39it/s]



Q: What do people use to absorb extra ink from a fountain pen? Answer Choices: (A) shirt pocket (B) calligrapher’s hand (C) inkwell (D) desk drawer (E) blotter

A: The answer must be an item that can absorb ink. Of the above choices, only blotters are used to absorb ink. So the answer is (E).

Q: What home entertainment equipment requires cable? Answer Choices: (A) radio shack (B) substation (C) television (D) cabinet

A: The answer must require cable. Of the above choices, only television requires cable. So the answer is (C).

Q: The fox walked from the city into the forest, what was it looking for? Answer Choices: (A) pretty flowers (B) hen house (C) natural habitat (D) storybook

A: The answer must be something in the forest. Of the above choices, only natural habitat is in the forest. So the answer is (B).

Q: Sammy wanted to go to where the people were. Where might he go? Answer Choices: (A) populated areas (B) race track (C) desert (D) apartment (E) roadblock

A: The answer mus

  0%|          | 2/1221 [00:00<08:25,  2.41it/s]



Q: What do people use to absorb extra ink from a fountain pen? Answer Choices: (A) shirt pocket (B) calligrapher’s hand (C) inkwell (D) desk drawer (E) blotter

A: The answer must be an item that can absorb ink. Of the above choices, only blotters are used to absorb ink. So the answer is (E).

Q: What home entertainment equipment requires cable? Answer Choices: (A) radio shack (B) substation (C) television (D) cabinet

A: The answer must require cable. Of the above choices, only television requires cable. So the answer is (C).

Q: The fox walked from the city into the forest, what was it looking for? Answer Choices: (A) pretty flowers (B) hen house (C) natural habitat (D) storybook

A: The answer must be something in the forest. Of the above choices, only natural habitat is in the forest. So the answer is (B).

Q: Sammy wanted to go to where the people were. Where might he go? Answer Choices: (A) populated areas (B) race track (C) desert (D) apartment (E) roadblock

A: The answer mus

  0%|          | 3/1221 [00:01<08:23,  2.42it/s]

...

  0%|          | 4/1221 [00:01<08:22,  2.42it/s]



Q: What do people use to absorb extra ink from a fountain pen? Answer Choices: (A) shirt pocket (B) calligrapher’s hand (C) inkwell (D) desk drawer (E) blotter

A: The answer must be an item that can absorb ink. Of the above choices, only blotters are used to absorb ink. So the answer is (E).

Q: What home entertainment equipment requires cable? Answer Choices: (A) radio shack (B) substation (C) television (D) cabinet

A: The answer must require cable. Of the above choices, only television requires cable. So the answer is (C).

Q: The fox walked from the city into the forest, what was it looking for? Answer Choices: (A) pretty flowers (B) hen house (C) natural habitat (D) storybook

A: The answer must be something in the forest. Of the above choices, only natural habitat is in the forest. So the answer is (B).

Q: Sammy wanted to go to where the people were. Where might he go? Answer Choices: (A) populated areas (B) race track (C) desert (D) apartment (E) roadblock

A: The answer mus

  0%|          | 5/1221 [00:02<08:23,  2.41it/s]



Q: What do people use to absorb extra ink from a fountain pen? Answer Choices: (A) shirt pocket (B) calligrapher’s hand (C) inkwell (D) desk drawer (E) blotter

A: The answer must be an item that can absorb ink. Of the above choices, only blotters are used to absorb ink. So the answer is (E).

Q: What home entertainment equipment requires cable? Answer Choices: (A) radio shack (B) substation (C) television (D) cabinet

A: The answer must require cable. Of the above choices, only television requires cable. So the answer is (C).

Q: The fox walked from the city into the forest, what was it looking for? Answer Choices: (A) pretty flowers (B) hen house (C) natural habitat (D) storybook

A: The answer must be something in the forest. Of the above choices, only natural habitat is in the forest. So the answer is (B).

Q: Sammy wanted to go to where the people were. Where might he go? Answer Choices: (A) populated areas (B) race track (C) desert (D) apartment (E) roadblock

A: The answer mus

0.2

In [18]:


method = pb.PEMethod(method='emotion_prompt',
                        dataset=dataset_name,
                        verbose=True,  # if True, print the detailed prompt and response
                        prompt_id = 1  # for emotion_prompt
                        )

results = method.test(dataset,
                      model,
                      num_samples=5 # if don't set the num_samples, method will use all examples in the dataset
                      )

results


  0%|          | 0/1221 [00:00<?, ?it/s]

A revolving door is convenient for two direction travel, but it also serves as a security measure at a what? 

Answer Choices: (A) bank (B) library (C) department store (D) mall (E) new york

This is very important to my career.

Please output your answer at the end as ##<your answer (among A through E)>



<pad> A</s>


  0%|          | 3/1221 [00:00<02:02,  9.98it/s]

What do people aim to do at work? 

Answer Choices: (A) complete job (B) learn from each other (C) kill animals (D) wear hats (E) talk to each other

This is very important to my career.

Please output your answer at the end as ##<your answer (among A through E)>



<pad> A</s>

Where would you find magazines along side many other printed works? 

Answer Choices: (A) doctor (B) bookstore (C) market (D) train station (E) mortuary

This is very important to my career.

Please output your answer at the end as ##<your answer (among A through E)>



<pad> B</s>


  0%|          | 4/1221 [00:00<02:02,  9.97it/s]

Where are  you likely to find a hamburger? 

Answer Choices: (A) fast food restaurant (B) pizza (C) ground up dead cows (D) mouth (E) cow carcus

This is very important to my career.

Please output your answer at the end as ##<your answer (among A through E)>



<pad> A</s>


  0%|          | 5/1221 [00:00<02:19,  8.74it/s]

James was looking for a good place to buy farmland.  Where might he look? 

Answer Choices: (A) midwest (B) countryside (C) estate (D) farming areas (E) illinois

This is very important to my career.

Please output your answer at the end as ##<your answer (among A through E)>



<pad> (E)</s>


0.8